In [1]:
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String

In [2]:
rospy.init_node('liveproc')
listener = tf.TransformListener()
listener.waitForTransform("/odom", "/front_left_camera_link", rospy.Time(), rospy.Duration(2.0))
# pub = rospy.Publisher('chatter', String, queue_size = 100)
occ_map = None
debug = False
    
np.set_printoptions(threshold=sys.maxsize)
# Once a color image has been read, it passes through the 
# transform function. The transform function has two parts:
# pre-processing also known as pp which filters the image based
# of certain values in the hue, saturation, and value (hsv) 
# colorspace and the dbscan which clusters the processed image
# and classifies the clusters as either a buoy, a green buoy,
# a red buoy, or noise. No changes are needed.

def transform(img, angle):
    global debug
    start_time = time.time()
    bridge = cv_bridge.CvBridge()
    
    width = img.height
    height = img.width
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    
    
    
    
    
    
    overlay = True
    image = pp(img,width,height)
    img2 = dbscan(image, img, overlay,angle)
    if debug:
        print("Transform: ", time.time()-start_time)
    return img2

# The first function in transform is pp or preprocessing. The 
# first line truncates the lower half of the image. The reason
# this is because the plantoons from the wamv are always 
# blocking the camera vision and become noisy data. The
# function changes the image to the HSV colorspace and 
# filters out the image. No changes are needed. It returns 
# the filtered image in the BGR colorspace.The input is the
# color image.

def pp(image, width, height):
    global debug
    start_time = time.time()
    image = np.array(image[0:width][0:int(height*0.65)])
    #image = cv.GaussianBlur(image, (3,3), sigmaX=0.25, sigmaY=0.25)
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)
    if debug:
        print("Preprocessing: ", time.time()-start_time)
    return(image)

# The second function in transform is dbscan. The inputs are 
# the filtered colored image and the original color image
# read from the imread statement. The output is the original
# image.This is the end of this classification algorithm.

###########################################################

def dbscan(image, original, overlay, angle):
    global debug
    start_time = time.time()
# Part 1: Thresholding the image
##########################################################
# The reason we threshold the image before applying DBSCAN to 
# the points is because DBSCAN is a computational demanding 
# function. This part takes the filtered image, makes a 
# copy of it, and converts it into a grayscale image. The gray-
# scale image is thresholded at a value of 10. 0 is black and
# 255 is white. This takes the ~400,000 matrix points and 
# leaves only ~10,000 nonzero matrix points. 

    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)

# After thresholding the image, we pull out those nonzero
# points and assign it to the nonzero array variable. This
# array has dimensions of n rows and 2 columns. The first 
# column is the y coordinate and the second column is the x
# coordinate. These two arrays would be combined by 
# np.column_stack which merges the two arrays at their columns.

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))
#########################################################

# Creating the BDSCAN object and Sorting Array with Labels
#########################################################    
# After thresholding the filtered image and creating an
# array of nonzero coordinates, this algorithm applies a
# clustering algorithm that filters the image based off the
# array X. If X is empty, this function ends and output is 
# the original input. The first line creates the DBSCAN
# object with an epsilon of 3 and minimum samples of 20. 
# Eps refers to the size of the neighborhood and the 
# min_samples refers to the minimum number of samples required
# for that neighborhood to be considered a cluster. The user
# is free to change the two values for eps and min_samples. The
# .fit(X) applies the matrix points to the DBSCAN object.
# One of the parameters of the db scan object are the labels.
# This is invoked by the line labels = db.labels. This creates
# an array of labels that assigns a number to the image index.
# n clusters have labels that start at 0 and end with n-1.For 
# example, if an image has four clusters, the labels would
# include 0,1,2, and 3. -1 is dedicated for noise. 

    if (len(X) > 0):
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_

# The array variable combines the x and y coordinates and the 
# labels in a single array aligned by column. The array is
# sorted with the sort variable with the values in the third
# column or the labels. For example, an image with four
# clusters and noise would be labeled in order from (-1,0,1,2,3).
# The while loop iterates from the beginning of the sorted loop
# and deletes every instance of -1 or until the list is empty.
# This while loop is needed because the noise data is
# irrelevant to the cluster algorithm and the user does not 
# know how much noise there is in the image. The image could 
# have no noise or it could all be noise. The output of this
# section is an array with coordinates sorted together in 
# clusters.

        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])

        x = 0
        while (len(sort) > 0 and sort[x][2] == -1):
            del sort[x]
#########################################################

# Iterating through and classifying each cluster
#########################################################
# After iterating through sort, the program is either left with
# an empty array or an array of coordinates sorted and grouped
# by each cluster. The user needs the number of clusters. The 
# next three lines in the big if block does this. unique_labels
# sorts the set of labels and removes -1 if present. The next 
# two lines creates a copy of the sort array and removes the 
# labels column from the array.

        if (len(sort) > 0):
            unique_labels = sorted(set(labels))
            if (unique_labels[0] == -1):
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

# The program assigns three variables. The first one i iterates
# through sort and compare its label value to x for each 
# iteration until the very end of the sort array. The second
# x refers to the cluster that the program is currently class-
# ifying. It starts with 0 and continues classifying until it 
# has classified all clusters. The third variable is l and this
# array becomes all the points within a cluster. Every time the 
# variable i iterates through sort and is equal to the value of
# the current cluster x, the coordinates are appended to the 
# array l. Since this array is sorted, the first time the x 
# variable equals a different value- the cluster has been 
# accounted for. This array of l contains all the points within
# the cluster. This array and the filtered image gets passed
# through the color function. The result of the color function
# gets passed through the rectangle function with the array,
# filtered image, and original image. The x variable is 
# incremented for the next cluster and the l array is set equal 
# to zero. Once the while loop has iterated through all 
# clusters, the result of the rectangle function is set to the 
# original image and the original image is returned to the 
# transform function which is returned to the read function 
# which writes the image.

            i = 0
            x = 0
            l = []
            while (x < len(unique_labels)):
                if (i < len(sort) and sort[i][2] == x):
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
                    clr = color_hsv(np.array(l),image)
                    tb = rectangle(np.array(l),clr,image,original, overlay,angle)
                    l = []
            original = tb
    if debug:
        print("DBSCAN: ", time.time()-start_time)
    return original

############################################################

# For the color function, the points of the cluster and the 
# filtered image are passed through. In this function, the 
# average red, green, and blue pixel values are calculated
# from the image inside of the cluster. If the average value
# fits within a certain range, the string 'red' or 'green' is 
# returned. Otherwise, the function returns the string 'none'
# The user is free to change the range of the BGR values that
# determine whether the buoy is red or green.

def color(mat,image):
    sumred = 0
    sumgrn = 0
    sumblu = 0
    for x in range(len(mat)):
        sumred += image [mat[x][1]] [mat[x][0]] [2]
        sumgrn += image [mat[x][1]] [mat[x][0]] [1]
        sumblu += image [mat[x][1]] [mat[x][0]] [0]
    if ((sumred/len(mat)) > 80 and (sumred/len(mat)) < 180 and (sumgrn/len(mat)) > 30 and (sumgrn/len(mat)) < 115  and (sumblu/len(mat)) > 30 and (sumblu/len(mat)) < 110):
        return 'red'
    if ((sumred/len(mat)) > 30 and (sumred/len(mat)) < 80 and (sumgrn/len(mat)) > 60 and (sumgrn/len(mat)) < 180  and (sumblu/len(mat)) > 50 and (sumblu/len(mat)) < 130):
        return 'green'
    else:
        return 'none'

def color_hsv(mat, image):
    global debug
    start_time = time.time()
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    sumhue = 0
    sumsat = 0
    sumval = 0
    mat_add = 0
    for x in range(len(mat)):
        if(image [mat[x][1]] [mat[x][0]] [2] > 5):
            mat_add = mat_add+1
            sumval += image [mat[x][1]] [mat[x][0]] [2]
            sumsat += image [mat[x][1]] [mat[x][0]] [1]
            sumhue += image [mat[x][1]] [mat[x][0]] [0]
    if mat_add > 0:
        sat = sumsat/mat_add
        val = sumval/mat_add
        hue = sumhue/mat_add
        #return (hue, sat, val)
        if (sat > 5 and val > 5 and (hue < 60 or hue >= 170)):
            return 'red'
        if (sat > 5 and val > 5 and (70 < hue < 90)):
            return 'green'
        else:
            return (hue, sat, val)
    if debug:
        print("Color: ", time.time()-start_time)
    return None

# The rectangle function's parameters are the array of coordinates
# of the cluster, the color of the cluster, the filtered image,
# and the original image. The boundingRect function applies
# a non-rotated rectangle to the image points and returns four
# values in an array, the x and y coordinates of the leftmost
# corner and the width and height. These values are converted
# to ints. The font is set to Hershey Simple for the text and
# the tag is set to zero. The tag is useful in the classifica-
# tion of the buoy. The first classification of the buoy is 
# the proportion of the width and height. If the height is
# between 1.5 times the width and 3.5 times the width, the
# cluster is considered a buoy. If the cluster is given a 
# 'red' or 'green' color, it would be classified as a 'red buoy'
# or 'green buoy' respectively. Otherwise, it would simply be
# classified as a buoy. If any of these three options occur, 
# the tag is set to 1. If the height is not in that range
# mentioned above and the color is 'none', the cluster is
# classified as noise. The last function rectangle makes a 
# rectangle over the original image. The original image is sent
# back to the DBSCAN function.

def rectangle(mat,color,image,original, overlay, angle):
    global debug
    start_time = time.time()
    rect = cv.boundingRect(mat)
    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    tag = 0
    if overlay:
        if (width < height and 4*width > height and width*height > 45):
            #print(color)
            bearing = ((x+width/2.0)-640.0)/1280.0*80.0
            if(color =='green'):
                tag = 1
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
            elif(color =='red'):
                tag = 1
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
            else:
                tag = 1
                cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            cv.putText(original,str(bearing) + " degrees",(np.int0(x-width/2),np.int0(y-height*math.log(height,10))), font, 0.5,(19,69,139),2,cv.LINE_AA)
            cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = ((x+width/2.0)-640.0)/1280.0*80.0
            angle.append(bearing)
        if(tag == 0):
            image[y:y+height][x:x+width]=[0,0,0]
        return original, angle
    else:
        if (width < height and 4*width > height and width*height > 45):
            #print(color)
            bearing = ((x+width/2.0)-640.0)/1280.0*80.0
            if(color =='green'):
                tag = 1
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            elif(color =='red'):
                tag = 1
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            else:
                tag = 1
                cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            cv.putText(image,str(bearing) + " degrees",(np.int0(x-width/2),np.int0(y-height*math.log(height,10))), font, 0.5,(255,255,255),2,cv.LINE_AA)
            cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = ((x+width/2.0)-640.0)/1280.0*80.0
            angle.append(bearing)
        if(tag == 0):
            image[y:y+height][x:x+width]=[0,0,0]
            #cv.putText(image,'noise',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
    if debug:
        print("Rectangle: ",time.time()-start_time)
    return image, angle

In [3]:
def createLineIterator(P1, P2, img):
    global debug
    start_time = time.time()
# """
# Produces and array that consists of the coordinates and intensities of
# each pixel in a line between two points

# Parameters:
#     -P1: a numpy array that consists of the coordinate of the first point (x,y)
#     -P2: a numpy array that consists of the coordinate of the second point (x,y)
#     -img: the image being processed

# Returns:
#     -it: a numpy array that consists of the coordinates and intensities of each
#     pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
# """
   #define local variables for readability
    #print img.shape
    image_height = img.shape[0]
    image_width = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]
    
   #difference and absolute difference between points
   #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(dYa,dXa),3))
    itbuffer.fill(np.nan)

   #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

   #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<image_width) & (colY<image_height)]

   #Get intensities from img ndarray
    itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]
    if debug:
        print("Lines: ", time.time()-start_time)
    return itbuffer

In [4]:
def map_filter(p_map, trans, robot_yaw, yaw_offset):
    global debug
    start_time = time.time()
    map_array = p_map.data
    origin = p_map.info.origin.position
    map_width = p_map.info.width
    map_height = p_map.info.height
    map_np = np.reshape(np.array(map_array),(map_height,map_width))
    

    print_to_file = False
    robot_pos_x = trans[0]-origin.x
    robot_pos_y = trans[1]-origin.y
    if (robot_yaw < 0):
        robot_yaw = robot_yaw + 360
    i = 0
    f = None
    while (i < len(yaw_offset)):
        buoy_yaw = robot_yaw - yaw_offset[i]
        if (math.fabs(buoy_yaw) > 180):
            buoy_yaw = buoy_yaw + 360
        camera_range = 30
        point2_x = camera_range * math.sin(buoy_yaw)
        point2_y = camera_range * math.cos(buoy_yaw)
        point1 = np.array([round(robot_pos_x), round(robot_pos_y)])
        point2 = np.array([round(point2_x), round(point2_y)])
        sliced = createLineIterator(point1.astype(int),point2.astype(int),map_np)
        if print_to_file:
            f = open("odom.txt", "w+")
            f.write("slice " + str(i) + ": " + str(sliced) + "\n")
        i+=1
    if print_to_file:
        f.write("map: " + str(map_np))
        f.write("translation:"+ str(trans))
        f.write("robot yaw:"+ str(robot_yaw))
        f.write("bearing:"+ str(yaw_offset))
        f.close()
#         print robot_yaw, buoy_yaw
#         rospy.sleep(1)
    if debug:
        print("Filter: ", time.time()-start_time)

In [5]:
# For whatever reason, trying to display /anything/ generated by this function
# causes the entire kernel to lag so... look into that
def get_map(p_map):
    map_array = p_map.data
    map_width = p_map.info.width
    map_height = p_map.info.height
    map_np = np.reshape(np.array(map_array),(map_height,map_width))
    
    map_img = np.zeros((map_height, map_width, 3))
    map_img[map_np == -1] = [255,0,0]
    map_img[map_np == 0] = [0,255,0]
    map_img[map_np == 100] = [0,0,255]
    return(map_img)

In [6]:
def callback(rect):
    global debug
    global occ_map
    #print "right before"
    start_time = time.time()
    display_rect = True
    angle = []
#     bridge = cv_bridge.CvBridge()

#     r_width = rect.height
#     r_height = rect.width
#     rect = bridge.imgmsg_to_cv2(rect, desired_encoding="passthrough")
#     rect = cv.cvtColor(rect, cv.COLOR_RGB2BGR)
    rect, bearings = transform(rect,angle)

    if(display_rect):
        cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
        cv.imshow("stereo/left/image_rect_color", rect)
        cv.waitKey(1)

    #print "inside"
#     map_image_latest = get_map(p_map)
#     if map_image_latest.size > 0:
#         cv.namedWindow("map",cv.WINDOW_NORMAL)
#         cv.imshow("map", map_image_latest)
#         cv.waitKey(1)
    if debug:
        print("Up to listener: ", time.time()-start_time)
#     global listener 
# #     rate = rospy.Rate(0.05)
#     if not rospy.is_shutdown():       
#         try:
#             now = rospy.Time.now()
#             listener.waitForTransform("/odom", "/front_left_camera_link", now, rospy.Duration(2.0))
#             (trans,rot) = listener.lookupTransform("/odom", "/front_left_camera_link", now)
#             euler = tf.transformations.euler_from_quaternion(rot)
#             pixel_line = map_filter(occ_map, trans, math.degrees(euler[2]), bearings)

#         except tf.LookupException as l:
#             print l
#         except tf.ConnectivityException as c:
#             print c
#         except tf.ExtrapolationException as e:
#             print e
#         except Exception as ex:
#             print ex
#     global pub
#     string = "hello %s" % rospy.get_time()
#     pub.publish(string)
    timing = True
    if debug or timing:
        print ("Callback: ",time.time()-start_time)

In [7]:
def map_callback(p_map):
    global occ_map
    occ_map = p_map

In [8]:
if __name__ == '__main__':
#     proj_map = rospy.Subscriber("projected_map", Map, map_callback)
    rect = rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
#     rect = message_filters.Subscriber("stereo/left/image_rect_color", Image)
#     p_map = message_filters.Subscriber("projected_map", Map)


#     ts = message_filters.TimeSynchronizer([rect, p_map],10)
#     ts.registerCallback(callback)
#     print "callback"
#     global occ_map
#     print(occ_map)
    rospy.spin()
    

('Callback: ', 0.2165849208831787)
('Callback: ', 0.10641312599182129)
('Callback: ', 0.10161709785461426)
('Callback: ', 0.11081814765930176)
('Callback: ', 0.15369105339050293)
('Callback: ', 0.10994791984558105)
('Callback: ', 0.0963129997253418)
('Callback: ', 0.1096501350402832)
('Callback: ', 0.11284399032592773)
('Callback: ', 0.09885001182556152)
('Callback: ', 0.10915899276733398)
('Callback: ', 0.10994219779968262)
('Callback: ', 0.11002206802368164)
('Callback: ', 0.10015416145324707)
('Callback: ', 0.10982608795166016)
('Callback: ', 0.1105489730834961)
('Callback: ', 0.11426091194152832)
('Callback: ', 0.1063239574432373)
('Callback: ', 0.1163320541381836)
('Callback: ', 0.10883283615112305)
('Callback: ', 0.11694502830505371)
('Callback: ', 0.11881685256958008)
('Callback: ', 0.1158609390258789)
('Callback: ', 0.11347317695617676)
('Callback: ', 0.11347818374633789)
('Callback: ', 0.11524200439453125)
('Callback: ', 0.129349946975708)
('Callback: ', 0.11675906181335449)
(

('Callback: ', 0.09503006935119629)
('Callback: ', 0.09948897361755371)
('Callback: ', 0.09413385391235352)
('Callback: ', 0.10917997360229492)
('Callback: ', 0.09575414657592773)
('Callback: ', 0.10383296012878418)
('Callback: ', 0.09743785858154297)
('Callback: ', 0.11230111122131348)
('Callback: ', 0.10561013221740723)
('Callback: ', 0.12456607818603516)
('Callback: ', 0.10289406776428223)
('Callback: ', 0.11267805099487305)
('Callback: ', 0.11173200607299805)
('Callback: ', 0.11552190780639648)
('Callback: ', 0.11325502395629883)
('Callback: ', 0.11880207061767578)
('Callback: ', 0.11170387268066406)
('Callback: ', 0.11495804786682129)
('Callback: ', 0.12466597557067871)
('Callback: ', 0.10641980171203613)
('Callback: ', 0.09986495971679688)
('Callback: ', 0.11393094062805176)
('Callback: ', 0.10950803756713867)
('Callback: ', 0.10634398460388184)
('Callback: ', 0.10191082954406738)
('Callback: ', 0.11571216583251953)
('Callback: ', 0.10916900634765625)
('Callback: ', 0.11445116996

('Callback: ', 0.10178709030151367)
('Callback: ', 0.10254907608032227)
('Callback: ', 0.09888005256652832)
('Callback: ', 0.0992879867553711)
('Callback: ', 0.09340214729309082)
('Callback: ', 0.09662413597106934)
('Callback: ', 0.1024179458618164)
('Callback: ', 0.0944981575012207)
('Callback: ', 0.09437990188598633)
('Callback: ', 0.09296607971191406)
('Callback: ', 0.09374523162841797)
('Callback: ', 0.0848379135131836)
('Callback: ', 0.08420991897583008)
('Callback: ', 0.08108305931091309)
('Callback: ', 0.08244609832763672)
('Callback: ', 0.08698606491088867)
('Callback: ', 0.07740902900695801)
('Callback: ', 0.07293105125427246)
('Callback: ', 0.08372187614440918)
('Callback: ', 0.07230591773986816)
('Callback: ', 0.08776187896728516)
('Callback: ', 0.07676196098327637)
('Callback: ', 0.08172202110290527)
('Callback: ', 0.08643698692321777)
('Callback: ', 0.08022308349609375)
('Callback: ', 0.07900309562683105)
('Callback: ', 0.08343887329101562)
('Callback: ', 0.080550909042358

('Callback: ', 0.10065698623657227)
('Callback: ', 0.1090550422668457)
('Callback: ', 0.1063680648803711)
('Callback: ', 0.10088515281677246)
('Callback: ', 0.11412692070007324)
('Callback: ', 0.11668086051940918)
('Callback: ', 0.11021590232849121)
('Callback: ', 0.10291314125061035)
('Callback: ', 0.10192012786865234)
('Callback: ', 0.0955190658569336)
('Callback: ', 0.09578514099121094)
('Callback: ', 0.09739398956298828)
('Callback: ', 0.09488797187805176)
('Callback: ', 0.08713293075561523)
('Callback: ', 0.09806609153747559)
('Callback: ', 0.0857090950012207)
('Callback: ', 0.09862804412841797)
('Callback: ', 0.08229994773864746)
('Callback: ', 0.09307003021240234)
('Callback: ', 0.09620308876037598)
('Callback: ', 0.08761191368103027)
('Callback: ', 0.09883284568786621)
('Callback: ', 0.09092187881469727)
('Callback: ', 0.09206295013427734)
('Callback: ', 0.08864212036132812)
('Callback: ', 0.08336210250854492)
('Callback: ', 0.08845114707946777)
('Callback: ', 0.093071937561035

('Callback: ', 0.1114649772644043)
('Callback: ', 0.10722804069519043)
('Callback: ', 0.1167287826538086)
('Callback: ', 0.12176990509033203)
('Callback: ', 0.10765409469604492)
('Callback: ', 0.11722207069396973)
('Callback: ', 0.0971219539642334)
('Callback: ', 0.11165094375610352)
('Callback: ', 0.1047520637512207)
('Callback: ', 0.11142897605895996)
('Callback: ', 0.10548901557922363)
('Callback: ', 0.10111713409423828)
('Callback: ', 0.10735607147216797)
('Callback: ', 0.12007880210876465)
('Callback: ', 0.1143648624420166)
('Callback: ', 0.10699009895324707)
('Callback: ', 0.11703205108642578)
('Callback: ', 0.11404609680175781)
('Callback: ', 0.10796594619750977)
('Callback: ', 0.10593509674072266)
('Callback: ', 0.11605691909790039)
('Callback: ', 0.09780502319335938)
('Callback: ', 0.10578107833862305)
('Callback: ', 0.09850406646728516)
('Callback: ', 0.10065603256225586)
('Callback: ', 0.11163997650146484)
('Callback: ', 0.10193085670471191)
('Callback: ', 0.1131651401519775

...

('Callback: ', 0.1136770248413086)
('Callback: ', 0.1028299331665039)
('Callback: ', 0.10305118560791016)
('Callback: ', 0.10009002685546875)
('Callback: ', 0.09943699836730957)
('Callback: ', 0.10883593559265137)
('Callback: ', 0.135300874710083)
('Callback: ', 0.12146997451782227)
('Callback: ', 0.13608193397521973)
('Callback: ', 0.10583019256591797)
('Callback: ', 0.11335396766662598)
('Callback: ', 0.10716104507446289)
('Callback: ', 0.11300897598266602)
('Callback: ', 0.10153508186340332)
('Callback: ', 0.10320091247558594)
('Callback: ', 0.10580110549926758)
('Callback: ', 0.08723592758178711)
('Callback: ', 0.09058880805969238)
('Callback: ', 0.09659695625305176)
('Callback: ', 0.1027381420135498)
('Callback: ', 0.09771108627319336)
('Callback: ', 0.09661102294921875)
('Callback: ', 0.09827208518981934)
('Callback: ', 0.0941309928894043)
('Callback: ', 0.09122896194458008)
('Callback: ', 0.09522294998168945)
('Callback: ', 0.09571504592895508)
('Callback: ', 0.09951615333557129

('Callback: ', 0.0971989631652832)
('Callback: ', 0.09678101539611816)
('Callback: ', 0.0903630256652832)
('Callback: ', 0.0999908447265625)
('Callback: ', 0.08109688758850098)
('Callback: ', 0.09974384307861328)
('Callback: ', 0.09420204162597656)
('Callback: ', 0.09102702140808105)
('Callback: ', 0.09584593772888184)
('Callback: ', 0.08822298049926758)
('Callback: ', 0.11070895195007324)
('Callback: ', 0.10724210739135742)
('Callback: ', 0.10392904281616211)
('Callback: ', 0.1210639476776123)
('Callback: ', 0.1082148551940918)
('Callback: ', 0.12464499473571777)
('Callback: ', 0.10807490348815918)
('Callback: ', 0.11871099472045898)
('Callback: ', 0.12142109870910645)
('Callback: ', 0.13010501861572266)
('Callback: ', 0.12247800827026367)
('Callback: ', 0.1272449493408203)
('Callback: ', 0.12359189987182617)
('Callback: ', 0.1269688606262207)
('Callback: ', 0.12946105003356934)
('Callback: ', 0.12094497680664062)
('Callback: ', 0.1087489128112793)
('Callback: ', 0.11609506607055664)


('Callback: ', 0.11908507347106934)
('Callback: ', 0.11864686012268066)
('Callback: ', 0.12880301475524902)
('Callback: ', 0.11430215835571289)
('Callback: ', 0.11178898811340332)
('Callback: ', 0.12242007255554199)
('Callback: ', 0.1354072093963623)
('Callback: ', 0.1343998908996582)
('Callback: ', 0.125715970993042)
('Callback: ', 0.13298296928405762)
('Callback: ', 0.10832810401916504)
('Callback: ', 0.1435987949371338)
('Callback: ', 0.09459400177001953)
('Callback: ', 0.08883404731750488)
('Callback: ', 0.09506011009216309)
('Callback: ', 0.09301209449768066)
('Callback: ', 0.09188604354858398)
('Callback: ', 0.09208893775939941)
('Callback: ', 0.09093713760375977)
('Callback: ', 0.09061884880065918)
('Callback: ', 0.09340119361877441)
('Callback: ', 0.09097814559936523)
('Callback: ', 0.08568406105041504)
('Callback: ', 0.11276698112487793)
('Callback: ', 0.09872817993164062)
('Callback: ', 0.10097384452819824)
('Callback: ', 0.10245299339294434)
('Callback: ', 0.1055090427398681

('Callback: ', 0.13035202026367188)
('Callback: ', 0.1279280185699463)
('Callback: ', 0.14514493942260742)
('Callback: ', 0.13760805130004883)
('Callback: ', 0.1346900463104248)
('Callback: ', 0.12207317352294922)
('Callback: ', 0.12300395965576172)
('Callback: ', 0.11443209648132324)
('Callback: ', 0.11066818237304688)
('Callback: ', 0.12008810043334961)
('Callback: ', 0.11042904853820801)
('Callback: ', 0.10489702224731445)
('Callback: ', 0.10760807991027832)
('Callback: ', 0.0908961296081543)
('Callback: ', 0.11032390594482422)
('Callback: ', 0.10625386238098145)
('Callback: ', 0.10615110397338867)
('Callback: ', 0.10738301277160645)
('Callback: ', 0.09557008743286133)
('Callback: ', 0.09881711006164551)
('Callback: ', 0.09337997436523438)
('Callback: ', 0.1034400463104248)
('Callback: ', 0.0905148983001709)
('Callback: ', 0.09540605545043945)
('Callback: ', 0.09309697151184082)
('Callback: ', 0.11137580871582031)
('Callback: ', 0.10650396347045898)
('Callback: ', 0.1164751052856445

('Callback: ', 0.08221697807312012)
('Callback: ', 0.09553909301757812)
('Callback: ', 0.09789299964904785)
('Callback: ', 0.11022806167602539)
('Callback: ', 0.14217495918273926)
('Callback: ', 0.10599088668823242)
('Callback: ', 0.09923386573791504)
('Callback: ', 0.10218095779418945)
('Callback: ', 0.10804104804992676)
('Callback: ', 0.10600996017456055)
('Callback: ', 0.10672903060913086)
('Callback: ', 0.10883307456970215)
('Callback: ', 0.09607887268066406)
('Callback: ', 0.09642815589904785)
('Callback: ', 0.09918880462646484)
('Callback: ', 0.12336301803588867)
('Callback: ', 0.11168313026428223)
('Callback: ', 0.12295007705688477)
('Callback: ', 0.12944316864013672)
('Callback: ', 0.12805700302124023)
('Callback: ', 0.13509082794189453)
('Callback: ', 0.1273479461669922)
('Callback: ', 0.11390399932861328)
('Callback: ', 0.11426806449890137)
('Callback: ', 0.13700008392333984)
('Callback: ', 0.10855889320373535)
('Callback: ', 0.10095810890197754)
('Callback: ', 0.118149042129